# Downloading and Exploring the data

In [1]:
import pandas as pd
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
#reading the data as dataframe


data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header =0 )
col = ['Postcode', 'Borough','Neighbourhood']
for data in data:
    current_col = data.columns.values[:3]
    if len(current_col) != len(col):
        continue
    if all(current_col == col):
        break
        

In [3]:
data.head(11)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [4]:
#to check the number of columns in dataframe

data.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [5]:
#cleaning the data
data.drop(data.loc[data['Neighbourhood']=='Not assigned'].index, inplace=True)
data.reset_index(drop=True, inplace=True)

data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,Rouge
7,M1B,Scarborough,Malvern
8,M3B,North York,Don Mills North
9,M4B,East York,Woodbine Gardens


In [6]:
# Processing the data
data['Neighbourhood'].replace('Not assigned', data['Borough'], inplace=True)
data.head(11)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,Rouge
7,M1B,Scarborough,Malvern
8,M3B,North York,Don Mills North
9,M4B,East York,Woodbine Gardens


In [7]:
# Creating a new dataframe with merged Neighbourhood
df = pd.DataFrame(data.groupby(['Postcode','Borough'], sort = False)['Neighbourhood'].apply(",".join))
df.reset_index(inplace = True)
df.head(11)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M1B,Scarborough,"Rouge,Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens,Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M6B,North York,Glencairn


In [8]:
# Number of rows and columns
df.shape

(102, 3)

# Generating latitude and longitude for each neighbourhood

In [9]:
file = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df1 = pd.read_csv(file)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
new_df = df.sort_values(by = 'Postcode')
new_df = pd.DataFrame(new_df)
new_df['Latitude'] = df1['Latitude']
new_df['Longitude'] = df1['Longitude']

new_df.set_index('Postcode',inplace = True)
new_df.reset_index(inplace = True)
new_df.head(11)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.744734,-79.239476
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.750072,-79.295849
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.803762,-79.363452
3,M1G,Scarborough,Woburn,43.789053,-79.408493
4,M1H,Scarborough,Cedarbrae,43.753259,-79.329656
5,M1J,Scarborough,Scarborough Village,43.739015,-79.506944
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.676357,-79.293031
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.659526,-79.340923
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.679563,-79.377529
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.657952,-79.387383


# Analysing the data

In [11]:
#finding the latitude and longitude values for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Canada")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [12]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

In [14]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [15]:
#creating foursquare credentials
CLIENT_ID = 'RXQMOAB1ZL3GKWN4LKOFIEVOPBS5GIISYJMUNTSPPRJ1D32X' 
CLIENT_SECRET = 'DLN0KYP0AMGB5PLPF3INC4SKKJGWCNLOKKM3TFTG1C2QHF22' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RXQMOAB1ZL3GKWN4LKOFIEVOPBS5GIISYJMUNTSPPRJ1D32X
CLIENT_SECRET:DLN0KYP0AMGB5PLPF3INC4SKKJGWCNLOKKM3TFTG1C2QHF22


In [16]:
#Creating new dataframe for clustering

toronto_data = new_df[new_df['Borough'].str.contains('Toronto').reset_index(drop=True)]
toronto_data.head()
toronto_data.set_index('Postcode',inplace = True)
toronto_data.reset_index(inplace = True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.778517,-79.346556
1,M4K,East Toronto,"The Danforth West,Riverdale",43.685347,-79.338106
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.715383,-79.405678
3,M4M,East Toronto,Studio District,43.654260,-79.360636
4,M4N,Central Toronto,Lawrence Park,43.647177,-79.381576


In [17]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] 
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] 

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.7785175, -79.3465557.


In [18]:
LIMIT = 50
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=RXQMOAB1ZL3GKWN4LKOFIEVOPBS5GIISYJMUNTSPPRJ1D32X&client_secret=DLN0KYP0AMGB5PLPF3INC4SKKJGWCNLOKKM3TFTG1C2QHF22&v=20180605&ll=43.7785175,-79.3465557&radius=500&limit=50'

In [19]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5e3b4d3677af0300280d763e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 67,
  'suggestedBounds': {'ne': {'lat': 43.7830175045, 'lng': -79.34033481358881},
   'sw': {'lat': 43.7740174955, 'lng': -79.35277658641118}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e848fbb5c5c9240de8e6a80',
       'name': 'The LEGO Store',
       'location': {'address': '1800 Sheppard Ave E',
        'crossStreet': 'at Don Mills Rd',
        'lat': 43.77820727238842,
        'lng': -79.34348299621146,
        'labeledLatLngs': [{'label': 'display',
          

In [20]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
    
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [24]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
1,The Beaches,43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery
2,The Beaches,43.778517,-79.346556,SilverCity Fairview Mall Cinemas,43.778681,-79.344085,Movie Theater
3,The Beaches,43.778517,-79.346556,CF Fairview Mall,43.777994,-79.343665,Shopping Mall
4,The Beaches,43.778517,-79.346556,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant


In [25]:

updated_df = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


updated_df['Neighbourhood'] = toronto_venues['Neighbourhood'] 


fixed_columns = [updated_df.columns[-1]] + list(updated_df.columns[:-1])
updated_df = updated_df[fixed_columns]

updated_df.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = updated_df.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
1,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.090909,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
5,"Cabbagetown,St. James Town",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
6,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
8,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
9,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighbourhood_venues_df = pd.DataFrame(columns=columns)
neighbourhood_venues_df['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_df.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhood_venues_df.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Caribbean Restaurant,Massage Studio,Bar,Miscellaneous Shop,Construction & Landscaping,Deli / Bodega,Discount Store,Diner,Dessert Shop
1,Berczy Park,Chinese Restaurant,Bank,Café,Japanese Restaurant,Yoga Studio,Deli / Bodega,Doner Restaurant,Dog Run,Discount Store,Diner
2,"Brockton,Exhibition Place,Parkdale Village",Park,Sandwich Place,Sushi Restaurant,Movie Theater,Pet Store,Pizza Place,Pub,Liquor Store,Burrito Place,Burger Joint
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Discount Store,Pizza Place,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant,Cuban Restaurant,Diner,Dessert Shop,Department Store
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Hotel,American Restaurant,Intersection,Sandwich Place,Burrito Place,Gym,Middle Eastern Restaurant,Fried Chicken Joint,Cosmetics Shop


# Creating clusters based on neighbourhood

In [40]:
kclusters = 6

toronto_neighbourhood_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_neighbourhood_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 1], dtype=int32)

In [42]:

neighbourhood_venues_df.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = toronto_data

df_merged = df_merged.join(neighbourhood_venues_df.set_index('Neighbourhood'), on='Neighbourhood')

df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Juice Bar,Tea Room,Cosmetics Shop,Food Court,Japanese Restaurant,Bakery,Bank
1,M4K,East Toronto,"The Danforth West,Riverdale",43.685347,-79.338106,0,Coffee Shop,Metro Station,Convenience Store,Intersection,Park,Cuban Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Park,Chinese Restaurant,Mexican Restaurant,Café,Restaurant,Salon / Barbershop,Dessert Shop
3,M4M,East Toronto,Studio District,43.654260,-79.360636,0,Coffee Shop,Pub,Café,Bakery,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Yoga Studio,Spa
4,M4N,Central Toronto,Lawrence Park,43.647177,-79.381576,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Deli / Bodega,Gastropub,Steakhouse,Gym / Fitness Center,Italian Restaurant,Beer Bar


In [44]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clt = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clt)
       
map_clt

# Segmentation of Neighbourhood

# First cluster 

In [56]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Juice Bar,Tea Room,Cosmetics Shop,Food Court,Japanese Restaurant,Bakery,Bank
1,"The Danforth West,Riverdale",0,Coffee Shop,Metro Station,Convenience Store,Intersection,Park,Cuban Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega
2,"The Beaches West,India Bazaar",0,Coffee Shop,Clothing Store,Fast Food Restaurant,Park,Chinese Restaurant,Mexican Restaurant,Café,Restaurant,Salon / Barbershop,Dessert Shop
3,Studio District,0,Coffee Shop,Pub,Café,Bakery,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Yoga Studio,Spa
4,Lawrence Park,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Deli / Bodega,Gastropub,Steakhouse,Gym / Fitness Center,Italian Restaurant,Beer Bar
5,Davisville North,0,Café,Bar,Bookstore,Bakery,Japanese Restaurant,Restaurant,Chinese Restaurant,Noodle House,Sandwich Place,Pub
7,Davisville,0,Breakfast Spot,Nightclub,Coffee Shop,Café,Furniture / Home Store,Gym,Burrito Place,Restaurant,Stadium,Bar
8,"Moore Park,Summerhill East",0,Bar,Mexican Restaurant,Thai Restaurant,Café,Park,Fried Chicken Joint,Gastropub,Flea Market,Grocery Store,Fast Food Restaurant
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,Coffee Shop,Park,Gym,Yoga Studio,Fast Food Restaurant,Salad Place,Portuguese Restaurant,Mexican Restaurant,Liquor Store,Juice Bar
11,"Cabbagetown,St. James Town",0,Coffee Shop,Convenience Store,Liquor Store,Beer Store,Cosmetics Shop,Café,Pizza Place,Concert Hall,Deli / Bodega,College Auditorium


# Second cluster 

In [57]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Church and Wellesley,1,Convenience Store,Yoga Studio,Curling Ice,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


# Third cluster

In [58]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Stn A PO Boxes 25 The Esplanade,2,Locksmith,Baseball Field,Yoga Studio,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Department Store


# Fourth cluster

In [59]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North Toronto West,3,Asian Restaurant,Pizza Place,Japanese Restaurant,Pub,Park,College Gym,Comfort Food Restaurant,Discount Store,Diner,Dessert Shop
10,Rosedale,3,Park,River,College Arts Building,Cuban Restaurant,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
17,Central Bay Street,3,Park,Convenience Store,Bar,Bank,Dance Studio,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
25,"Harbord,University of Toronto",3,Park,Basketball Court,Construction & Landscaping,Bakery,Dance Studio,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
31,"Dovercourt Village,Dufferin",3,Airport,Snack Place,Bus Stop,Park,Frozen Yogurt Shop,Fried Chicken Joint,Diner,Dessert Shop,Department Store,Deli / Bodega
35,"Parkdale,Roncesvalles",3,Park,Women's Store,Market,Fast Food Restaurant,Curling Ice,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


# Fifth cluster

In [62]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,St. James Town,4,Park,Playground,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice


# Sixth cluster

In [63]:
df_merged.loc[df_merged['Cluster Labels'] == 5, df_merged.columns[[2] + list(range(5, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Ryerson,Garden District",5,American Restaurant,Motel,Yoga Studio,Curling Ice,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
